In [1]:
import pandas as pd
import sqlite3

In [3]:
cnx = sqlite3.connect('foo.db')

In [5]:
df_customers = pd.read_sql_query("SELECT * FROM olist_customers_dataset", cnx)
df_geolocations = pd.read_sql_query("SELECT * FROM olist_geolocation_dataset", cnx)
df_orderitems = pd.read_sql_query("SELECT * FROM olist_order_items_dataset", cnx)
df_payments = pd.read_sql_query("SELECT * FROM olist_order_payments_dataset", cnx)
df_reviews = pd.read_sql_query("SELECT * FROM olist_order_reviews_dataset", cnx)
df_orders = pd.read_sql_query("SELECT * FROM olist_orders_dataset", cnx)
df_products = pd.read_sql_query("SELECT * FROM olist_products_dataset", cnx)
df_sellers = pd.read_sql_query("SELECT * FROM olist_sellers_dataset", cnx)

In [31]:
import pandas as pd

# Misalkan df_customers dan df_orders sudah didefinisikan sebelumnya
# df_customers = pd.read_csv('path_to_customers.csv')
# df_orders = pd.read_csv('path_to_orders.csv')

# Langkah 1: Menghitung jumlah order untuk setiap customer
order_counts = df_orders['customer_id'].value_counts().reset_index()
order_counts.columns = ['customer_id', 'order_count']

# Langkah 2: Menggabungkan dengan df_customers untuk mendapatkan customer_unique_id
order_counts = pd.merge(order_counts, df_customers[['customer_id', 'customer_unique_id']], on='customer_id', how='left')

# Langkah 3: Menentukan klasifikasi frekuensi order
def classify_frequency(count):
    if count < 2:
        return 'rendah'
    elif 2 <= count <= 5:
        return 'menengah'
    else:
        return 'tinggi'

order_counts['class'] = order_counts['order_count'].apply(classify_frequency)

# # Menampilkan hasil
# print(order_counts[['customer_unique_id', 'order_count', 'class']])


In [39]:
# Langkah 1: Menghitung jumlah order untuk setiap customer_id
order_counts = df_orders.groupby('customer_id').size().reset_index(name='order_count')

# Langkah 2: Menggabungkan dengan df_customers untuk mendapatkan customer_unique_id
order_counts = pd.merge(order_counts, df_customers[['customer_id', 'customer_unique_id']], on='customer_id', how='left')

# Langkah 3: Mengelompokkan berdasarkan customer_unique_id
final_counts = order_counts.groupby('customer_unique_id')['order_count'].sum().reset_index()


In [41]:
final_counts

,customer_unique_id,order_count
0,0000366f3b9a7992bf8c76cfdf3221e2,1
1,0000b849f77a49e4a4ce2b2a4ca5be3f,1
2,0000f46a3911fa3c0805444483337064,1
3,0000f6ccb0745a6a4b88665a16c9f078,1
4,0004aac84e0df4da2b147fca70cf8255,1
...,...,...
96091,fffcf5a5ff07b0908bd4e2dbc735a684,1
96092,fffea47cd6d3cc0a88bd621562a9d061,1
96093,ffff371b4d645b6ecea244b27531430a,1
96094,ffff5962728ec6157033ef9805bacc48,1


In [43]:
final_counts.sort_values('order_count')

,customer_unique_id,order_count
0,0000366f3b9a7992bf8c76cfdf3221e2,1
63580,a926cfc9bc7b082335de50450f48eec9,1
63579,a926aae38267e7f54e67de9b5775d0a5,1
63578,a92605ec492805540520d3a73aaeeb6e,1
63577,a925c3e5df82fdc6082f1383d2834998,1
...,...,...
76082,ca77025e7201e3b30c44b472ff346268,7
37797,6469f99c1f9dfae7733b25662e7f1782,7
10354,1b6c7548a2a1f9037c1fd3ddfed95f33,7
23472,3e43e6105506432c953e165fb2acf44c,9


In [71]:
final_counts['group'] = pd.qcut(final_counts['order_count'], 
                                    q=3, 
                                    labels=['Low', 'Medium', 'High'], 
                                    duplicates='drop')

ValueError: Bin labels must be one fewer than the number of bin edges

In [61]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Misalkan final_counts sudah didefinisikan sebelumnya
# final_counts = pd.read_csv('path_to_final_counts.csv')

# Langkah 1: Standarisasi data
scaler = StandardScaler()
final_counts['order_count_scaled'] = scaler.fit_transform(final_counts[['order_count']])

# Langkah 2: Mencari jumlah kluster yang ideal menggunakan Silhouette Score
silhouette_scores = []
range_n_clusters = range(2, 11)  # Mencoba dari 2 hingga 10 kluster

for n_clusters in range_n_clusters:
    kmeans = KMeans(n_clusters=3, random_state=42)
    cluster_labels = kmeans.fit_predict(final_counts[['order_count_scaled']])
    silhouette_avg = silhouette_score(final_counts[['order_count_scaled']], cluster_labels)
    silhouette_scores.append(silhouette_avg)

# Menentukan jumlah kluster dengan Silhouette Score tertinggi
optimal_n_clusters = range_n_clusters[silhouette_scores.index(max(silhouette_scores))]

# Langkah 3: Melakukan klasterisasi menggunakan K-Means dengan jumlah kluster optimal
kmeans = KMeans(n_clusters=optimal_n_clusters, random_state=42)
final_counts['cluster'] = kmeans.fit_predict(final_counts[['order_count_scaled']])

# Menampilkan hasil
# print(final_counts[['customer_unique_id', 'order_count', 'order_count_scaled', 'cluster']])



KeyboardInterrupt



In [59]:
final_counts.groupby('cluster').agg({'order_count':['min','max','mean','median','count']})

order_count                        
                min max  mean median  count
cluster                                    
0                 1   1   1.0    1.0  93099
1                 2   2   2.0    2.0   2745
2                 4   4   4.0    4.0     30
3                 3   3   3.0    3.0    203
4                17  17  17.0   17.0      1
5                 7   7   7.0    7.0      3
6                 5   5   5.0    5.0      8
7                 6   6   6.0    6.0      6
8                 9   9   9.0    9.0      1

In [57]:
final_counts[final_counts['cluster'] == 8]

,customer_unique_id,order_count,order_count_scaled,cluster
23472,3e43e6105506432c953e165fb2acf44c,9,37.154086,8


In [53]:
final_counts.value_counts('cluster')

cluster
0    93099
1     2745
3      203
2       30
6        8
7        6
5        3
4        1
8        1
Name: count, dtype: int64

In [33]:
order_counts.sort_values('order_count')

,customer_id,order_count,customer_unique_id,class
0,9ef432eb6251297304e76186b10a928d,1,7c396fd4830fd04220f754e42b4e5bff,rendah
66299,503740e9ca751ccdda7ba28e9ab8f608,1,80bb27c7c16e8f973207a5086ab329e2,rendah
66298,ed0271e0b7da060a393796590e7b737a,1,36edbb3fb164b1f16485364b6fb04c73,rendah
66297,9bdf08b4b3b52b5526ff42d37d47f222,1,932afa1e708222e5821dac9cd5db4cae,rendah
66296,f54a9f0e6b351c431402b8461ea51999,1,39382392765b6dc74812866ee5ee92a7,rendah
...,...,...,...,...
33141,a537aaf902d2f436eb7f44817154bb68,1,02029b32d4185a32a888c3f2d4d8839f,rendah
33140,2f4a42f9bb4b9a8cd402fa549df5c7fd,1,312823a3a38d7ad3046d8ba7122f93a8,rendah
33139,42b0671b440d1f2527bddd8b723e0a0c,1,b0854ff72fac3b922fde0115dd2c0a5f,rendah
33161,3803547f820b1e10b7ad9c7985f06753,1,bd7df6ff9e4a04991eea842883f79e9a,rendah


In [27]:
order_counts.value_counts('frequency_description')

KeyError: 'frequency_description'

In [6]:
import pandas as pd

# Misalkan df_customers dan df_orders sudah didefinisikan sebelumnya
# df_customers = pd.read_csv('path_to_customers.csv')
# df_orders = pd.read_csv('path_to_orders.csv')

# Langkah 1: Mengonversi kolom order_purchase_timestamp ke tipe datetime
df_orders['order_purchase_timestamp'] = pd.to_datetime(df_orders['order_purchase_timestamp'])

# Langkah 2: Melakukan merge antara df_orders dan df_customers
merged_df = pd.merge(df_orders, df_customers[['customer_id', 'customer_unique_id']], on='customer_id', how='left')

# Langkah 3: Mengelompokkan data berdasarkan customer_unique_id untuk mendapatkan tanggal transaksi terakhir
last_transaction_dates = merged_df.groupby('customer_unique_id')['order_purchase_timestamp'].max().reset_index()
last_transaction_dates.columns = ['customer_unique_id', 'last_transaction_date']

# Langkah 4: Menentukan tanggal transaksi akhir (tanggal terbaru dari seluruh transaksi)
final_transaction_date = merged_df['order_purchase_timestamp'].max()

# Langkah 5: Menambahkan kolom tanggal transaksi akhir dan menghitung selisih hari
last_transaction_dates['final_transaction_date'] = final_transaction_date
last_transaction_dates['days_difference'] = (last_transaction_dates['final_transaction_date'] - last_transaction_dates['last_transaction_date']).dt.days

# Menampilkan hasil
last_transaction_dates


,customer_unique_id,last_transaction_date,final_transaction_date,days_difference
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05-10 10:56:27,2018-10-17 17:30:18,160
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05-07 11:11:27,2018-10-17 17:30:18,163
2,0000f46a3911fa3c0805444483337064,2017-03-10 21:05:03,2018-10-17 17:30:18,585
3,0000f6ccb0745a6a4b88665a16c9f078,2017-10-12 20:29:41,2018-10-17 17:30:18,369
4,0004aac84e0df4da2b147fca70cf8255,2017-11-14 19:45:42,2018-10-17 17:30:18,336
...,...,...,...,...
96091,fffcf5a5ff07b0908bd4e2dbc735a684,2017-06-08 21:00:36,2018-10-17 17:30:18,495
96092,fffea47cd6d3cc0a88bd621562a9d061,2017-12-10 20:07:56,2018-10-17 17:30:18,310
96093,ffff371b4d645b6ecea244b27531430a,2017-02-07 15:49:16,2018-10-17 17:30:18,617
96094,ffff5962728ec6157033ef9805bacc48,2018-05-02 15:17:41,2018-10-17 17:30:18,168


In [7]:
last_transaction_dates[last_transaction_dates['days_difference'] == 744]

,customer_unique_id,last_transaction_date,final_transaction_date,days_difference
5521,0eb1ee9dba87f5b36b4613a65074337c,2016-10-02 22:07:52,2018-10-17 17:30:18,744
17870,2f64e403852e6893ae37485d5fcacdaf,2016-10-03 16:56:50,2018-10-17 17:30:18,744


In [ ]:
last_transaction_dates.value_counts('days_difference')

In [ ]:
# import pandas as pd

# # Misalkan df_customers dan df_orders sudah didefinisikan sebelumnya
# # df_customers = pd.read_csv('path_to_customers.csv')
# # df_orders = pd.read_csv('path_to_orders.csv')

# # Mengonversi kolom order_purchase_timestamp ke tipe datetime
# df_orders['order_purchase_timestamp'] = pd.to_datetime(df_orders['order_purchase_timestamp'])

# # Mengelompokkan data berdasarkan customer_id dan mendapatkan tanggal transaksi awal dan akhir
# df_grouped = df_orders.groupby('customer_id')['order_purchase_timestamp'].agg(['min', 'max']).reset_index()

# # Menghitung selisih hari antara transaksi awal dan akhir
# df_grouped['days_difference'] = (df_grouped['max'] - df_grouped['min']).dt.days

# # Menggabungkan dengan df_customers untuk mendapatkan customer_id
# result_df = pd.merge(df_customers[['customer_id']], df_grouped, on='customer_id', how='left')

# # Mengganti nama kolom untuk kejelasan
# result_df.columns = ['customer_id', 'first_transaction_date', 'last_transaction_date', 'days_difference']

# # Menampilkan hasil
# print(result_df)


In [ ]:
df_customers.to_csv('df_customers.csv', index=False)

In [ ]:
df_geolocations.to_csv('df_geolocations.csv', index=False)

In [ ]:
df_orderitems.to_csv('df_orderitems.csv', index=False)

In [ ]:
df_payments.to_csv('df_payments.csv', index=False)

In [ ]:
df_reviews.to_csv('df_reviews.csv', index=False)

In [ ]:
df_orders.to_csv('df_orders.csv', index=False)

In [ ]:
df_products.to_csv('df_products.csv', index=False)

In [ ]:
df_sellers.to_csv('df_sellers.csv', index=False)

In [ ]:
null_df_customer = df_customers.isnull().sum().sum()
null_df_geolocations = df_geolocations.isnull().sum().sum()
null_df_orderitems = df_orderitems.isnull().sum().sum()
null_df_payments = df_payments.isnull().sum().sum()
null_df_reviews = df_reviews.isnull().sum().sum()
null_df_orders = df_orders.isnull().sum().sum()
null_df_products = df_products.isnull().sum().sum()
null_df_sellers = df_sellers.isnull().sum().sum()

In [ ]:
print('Number of NaN values null_df_customer:', null_df_customer)
print('Number of NaN values null_df_geolocations:', null_df_geolocations)
print('Number of NaN values null_df_orderitems:', null_df_orderitems)
print('Number of NaN values null_df_payments:', null_df_payments)
print('Number of NaN values null_df_reviews:', null_df_reviews)
print('Number of NaN values null_df_orders:', null_df_orders)
print('Number of NaN values null_df_products:', null_df_products)
print('Number of NaN values null_df_sellers:', null_df_sellers)


In [ ]:
# Menggabungkan DataFrame
# Pertama, kita gabungkan df_orders dengan df_customers
merged_df = pd.merge(df_orders, df_customers, on='customer_id', how='inner')

# Menggabungkan dengan df_orderitems
merged_df = pd.merge(merged_df, df_orderitems, on='order_id', how='inner')

# Menggabungkan dengan df_payments
merged_df = pd.merge(merged_df, df_payments, on='order_id', how='inner')

# Menggabungkan dengan df_reviews
merged_df = pd.merge(merged_df, df_reviews, on='order_id', how='inner')

# Menggabungkan dengan df_products
merged_df = pd.merge(merged_df, df_products, on='product_id', how='inner')

# Menggabungkan dengan df_sellers
merged_df = pd.merge(merged_df, df_sellers, on='seller_id', how='inner')

# Menggabungkan dengan df_geolocations
merged_df = pd.merge(merged_df, df_geolocations, left_on='customer_zip_code_prefix', right_on='geolocation_zip_code_prefix', how='inner')


In [ ]:
# # Menggabungkan df_orders dengan df_orderitems berdasarkan order_id
# df = pd.merge(df_orderitems, df_orders, on='order_id', how='left')

# # Menggabungkan dengan df_payments berdasarkan order_id
# df = pd.merge(df, df_payments, on='order_id', how='left')

# # Menggabungkan dengan df_reviews berdasarkan order_id
# df = pd.merge(df, df_reviews, on='order_id', how='left')

# # Menggabungkan dengan df_products berdasarkan product_id
# df = pd.merge(df, df_products, on='product_id', how='left')

# # Menggabungkan dengan df_sellers berdasarkan seller_id
# df = pd.merge(df, df_sellers, on='seller_id', how='left')

# # Menggabungkan dengan df_customers berdasarkan customer_id
# df = pd.merge(df, df_customers, on='customer_id', how='left')

# # Menggabungkan dengan df_geolocations berdasarkan zip_code_prefix
# df = pd.merge(df, df_geolocations, left_on='customer_zip_code_prefix', right_on='geolocation_zip_code_prefix', how='left')

In [ ]:
mode_payment_type = df_clean['payment_type'].mode()[0]
df_clean['payment_type'].fillna(mode_payment_type, inplace=True)

In [ ]:
df_clean['payment_sequential'].fillna(method='ffill', inplace=True)